In [4]:
##%overwritefile
##%file:src/compile_with_goc.py
##%noruncode
    def compile_with_goc(self, source_filename, binary_filename, cflags=None, ldflags=None,env=None,magics=None):
        # cflags = ['-std=c89', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-Wdeclaration-after-statement', '-Wvla', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=iso9899:199409', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c11', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        outfile=binary_filename
        orig_cflags=cflags
        orig_ldflags=ldflags

        index=0
        for s in cflags:
            if s.startswith('-o'):
                if(len(s)>2):
                    outfile=s[2:]
                    del cflags[index]
                else:
                    outfile=cflags[cflags.index('-o')+1]
                    if outfile.startswith('-'):
                        outfile=binary_filename
                    del cflags[cflags.index('-o')+1]
                    del cflags[cflags.index('-o')]
            binary_filename=outfile
            index+=1
        args=[]
        if magics!=None and len(self.mymagics.addkey2dict(magics,'ccompiler'))>0:
            args = magics['ccompiler'] + orig_cflags +[source_filename] + orig_ldflags
        else:
            args = ['go','build']+ ['-o', binary_filename]+[source_filename] + cflags  + ldflags
        # self._log(''.join((' '+ str(s) for s in args))+"\n")
        return self.mymagics.create_jupyter_subprocess(args,env=env,magics=magics),binary_filename,args


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Go\src/compile_with_goc.py created successfully


In [5]:
##%overwritefile
##%file:src/go_exec_goc_.py
##%noruncode
    def _exec_goc_(self,source_filename,magics):
        self.mymagics._logln('Generating executable file')
        with self.mymagics.new_temp_file(suffix='.out') as binary_file:
            magics['status']='compiling'
            p,outfile,gcccmd = self.compile_with_goc(
                source_filename, 
                binary_file.name,
                self.mymagics.get_magicsSvalue(magics,'cflags'),
                self.mymagics.get_magicsSvalue(magics,'ldflags'),
                self.mymagics.get_magicsbykey(magics,'env'),
                magics=magics)
            returncode=p.wait_end(magics)
            p.write_contents()
            magics['status']='compiling'
            binary_file.name=os.path.join(os.path.abspath(''),outfile)
            if returncode != 0:  # Compilation failed
                self.mymagics._logln(''.join((str(s) for s in gcccmd))+"\n",3)
                self.mymagics._logln("Go build exited with code {}, the executable will not be executed".format(returncode),3)

                # delete source files before exit
                os.remove(source_filename)
                os.remove(binary_file.name)
        return p.returncode,binary_file.name


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Go\src/go_exec_goc_.py created successfully
